In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.feature_selection import RFE
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import os
print(os.listdir('/content/drive/MyDrive/idsprojectcsvs'))

['02-15-2018.csv', '02-21-2018.csv', '02-22-2018.csv', '03-01-2018.csv', '02-28-2018.csv', '02-16-2018.csv', '03-02-2018.csv', '02-14-2018.csv']


# Task
Load multiple datasets from Google Drive, compare their columns, and ensure they all have matching columns.

## Load data

### Subtask:
Load each of the datasets into separate pandas DataFrames.


**Reasoning**:
Define the folder path and load each CSV file into a dictionary of DataFrames.



In [8]:
folder_path = '/content/drive/MyDrive/idsprojectcsvs'
dfs = {}

for fname in os.listdir(folder_path):
    if fname.endswith('.csv'):
        path = os.path.join(folder_path, fname)
        dfs[fname] = pd.read_csv(path)

## Check columns

### Subtask:
Compare the columns of each DataFrame to identify any discrepancies.


**Reasoning**:
Compare the columns of each DataFrame to identify any discrepancies.



In [9]:
column_names = {name: df.columns.tolist() for name, df in dfs.items()}

all_columns = set()
for name, columns in column_names.items():
    all_columns.update(columns)

for name, columns in column_names.items():
    missing_columns = list(all_columns - set(columns))
    extra_columns = list(set(columns) - all_columns) # This will always be empty based on how all_columns is created, but included for completeness in comparison
    print(f"DataFrame: {name}")
    print(f"  Missing columns: {missing_columns}")
    print(f"  Extra columns: {extra_columns}")
    print("-" * 20)


DataFrame: 02-15-2018.csv
  Missing columns: []
  Extra columns: []
--------------------
DataFrame: 02-21-2018.csv
  Missing columns: []
  Extra columns: []
--------------------
DataFrame: 02-22-2018.csv
  Missing columns: []
  Extra columns: []
--------------------
DataFrame: 03-01-2018.csv
  Missing columns: []
  Extra columns: []
--------------------
DataFrame: 02-28-2018.csv
  Missing columns: []
  Extra columns: []
--------------------
DataFrame: 02-16-2018.csv
  Missing columns: []
  Extra columns: []
--------------------
DataFrame: 03-02-2018.csv
  Missing columns: []
  Extra columns: []
--------------------
DataFrame: 02-14-2018.csv
  Missing columns: []
  Extra columns: []
--------------------


In [10]:
for name, df in dfs.items():

    # clean column names
    df.columns = df.columns.str.strip().str.lower().str.replace(' ','_')

    # columns that should NOT be numerically converted
    non_numeric = ['label', 'protocol', 'day', 'timestamp']

    # convert all but the protected ones
    for c in df.columns:
        if c not in non_numeric:
            df[c] = pd.to_numeric(df[c].astype(str).str.strip(), errors='coerce')

    dfs[name] = df


## Verify columns

### Subtask:
After addressing the discrepancies, verify that all DataFrames now have matching columns.


**Reasoning**:
Iterate through the dfs dictionary and print the column names for each DataFrame to visually confirm they match.



In [11]:
for name, df in dfs.items():
    print(name, df.describe().T[['count','mean','std','min','max']].head(25))


02-15-2018.csv                       count          mean           std       min  \
dst_port          1048575.0  7.392499e+03  1.753869e+04  0.000000   
protocol          1048575.0  9.518465e+00  5.303721e+00  0.000000   
flow_duration     1048575.0  1.430739e+07  3.335503e+07  0.000000   
tot_fwd_pkts      1048575.0  5.145998e+00  3.379921e+01  1.000000   
tot_bwd_pkts      1048575.0  6.419784e+00  9.786280e+01  0.000000   
totlen_fwd_pkts   1048575.0  4.354262e+02  2.881334e+04  0.000000   
totlen_bwd_pkts   1048575.0  4.925884e+03  1.399474e+05  0.000000   
fwd_pkt_len_max   1048575.0  1.514397e+02  2.774302e+02  0.000000   
fwd_pkt_len_min   1048575.0  1.397992e+01  2.382720e+01  0.000000   
fwd_pkt_len_mean  1048575.0  4.472574e+01  6.052267e+01  0.000000   
fwd_pkt_len_std   1048575.0  5.079972e+01  9.133395e+01  0.000000   
bwd_pkt_len_max   1048575.0  3.305074e+02  4.938690e+02  0.000000   
bwd_pkt_len_min   1048575.0  3.365044e+01  5.568782e+01  0.000000   
bwd_pkt_len_mean  1

In [12]:
for name, df in dfs.items():
    # Find columns with only 1 unique value
    zero_cols = [c for c in df.columns if df[c].nunique() == 1]

    # Drop them from the DataFrame
    dfs[name] = df.drop(columns=zero_cols)

    print(f"Dropped from {name}: {zero_cols}")



Dropped from 02-15-2018.csv: ['bwd_psh_flags', 'fwd_urg_flags', 'bwd_urg_flags', 'cwe_flag_count', 'fwd_byts/b_avg', 'fwd_pkts/b_avg', 'fwd_blk_rate_avg', 'bwd_byts/b_avg', 'bwd_pkts/b_avg', 'bwd_blk_rate_avg']
Dropped from 02-21-2018.csv: ['bwd_psh_flags', 'fwd_urg_flags', 'bwd_urg_flags', 'cwe_flag_count', 'fwd_byts/b_avg', 'fwd_pkts/b_avg', 'fwd_blk_rate_avg', 'bwd_byts/b_avg', 'bwd_pkts/b_avg', 'bwd_blk_rate_avg']
Dropped from 02-22-2018.csv: ['bwd_psh_flags', 'fwd_urg_flags', 'bwd_urg_flags', 'cwe_flag_count', 'fwd_byts/b_avg', 'fwd_pkts/b_avg', 'fwd_blk_rate_avg', 'bwd_byts/b_avg', 'bwd_pkts/b_avg', 'bwd_blk_rate_avg']
Dropped from 03-01-2018.csv: ['bwd_psh_flags', 'bwd_urg_flags', 'fwd_byts/b_avg', 'fwd_pkts/b_avg', 'fwd_blk_rate_avg', 'bwd_byts/b_avg', 'bwd_pkts/b_avg', 'bwd_blk_rate_avg']
Dropped from 02-28-2018.csv: ['bwd_psh_flags', 'bwd_urg_flags', 'fwd_byts/b_avg', 'fwd_pkts/b_avg', 'fwd_blk_rate_avg', 'bwd_byts/b_avg', 'bwd_pkts/b_avg', 'bwd_blk_rate_avg']
Dropped from 02

In [13]:
print(dfs['02-14-2018.csv'].columns)
print(dfs['02-15-2018.csv'].columns)
print(dfs['02-16-2018.csv'].columns)
print(dfs['02-21-2018.csv'].columns)
print(dfs['02-22-2018.csv'].columns)
print(dfs['02-28-2018.csv'].columns)
print(dfs['03-01-2018.csv'].columns)
print(dfs['03-02-2018.csv'].columns)



Index(['dst_port', 'protocol', 'timestamp', 'flow_duration', 'tot_fwd_pkts',
       'tot_bwd_pkts', 'totlen_fwd_pkts', 'totlen_bwd_pkts', 'fwd_pkt_len_max',
       'fwd_pkt_len_min', 'fwd_pkt_len_mean', 'fwd_pkt_len_std',
       'bwd_pkt_len_max', 'bwd_pkt_len_min', 'bwd_pkt_len_mean',
       'bwd_pkt_len_std', 'flow_byts/s', 'flow_pkts/s', 'flow_iat_mean',
       'flow_iat_std', 'flow_iat_max', 'flow_iat_min', 'fwd_iat_tot',
       'fwd_iat_mean', 'fwd_iat_std', 'fwd_iat_max', 'fwd_iat_min',
       'bwd_iat_tot', 'bwd_iat_mean', 'bwd_iat_std', 'bwd_iat_max',
       'bwd_iat_min', 'fwd_psh_flags', 'fwd_header_len', 'bwd_header_len',
       'fwd_pkts/s', 'bwd_pkts/s', 'pkt_len_min', 'pkt_len_max',
       'pkt_len_mean', 'pkt_len_std', 'pkt_len_var', 'fin_flag_cnt',
       'syn_flag_cnt', 'rst_flag_cnt', 'psh_flag_cnt', 'ack_flag_cnt',
       'urg_flag_cnt', 'ece_flag_cnt', 'down/up_ratio', 'pkt_size_avg',
       'fwd_seg_size_avg', 'bwd_seg_size_avg', 'subflow_fwd_pkts',
       'subflow

In [14]:
for name, df in dfs.items():
    if 'timestamp' in df.columns:
        df = df.drop(columns=['timestamp'])
    dfs[name] = df


In [15]:
safe_numeric_cols = ['label','protocol']   # allow these to stay raw for now

for name in list(dfs.keys()):
    df = dfs[name]

    # normalize column names
    df.columns = df.columns.str.strip().str.lower().str.replace(' ','_')

    # convert each col to numeric except label/protocol
    for c in df.columns:
        if c not in safe_numeric_cols:
            df[c] = pd.to_numeric(df[c], errors='coerce')

    # replace inf -> nan -> median
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(df.median(numeric_only=True), inplace=True)

    # save back
    dfs[name] = df
    print(name, "done", df.shape)


02-15-2018.csv done (1048575, 69)
02-21-2018.csv done (1048575, 69)
02-22-2018.csv done (1048575, 69)
03-01-2018.csv done (331125, 71)
02-28-2018.csv done (613104, 71)
02-16-2018.csv done (1048575, 69)
03-02-2018.csv done (1048575, 69)
02-14-2018.csv done (1048575, 69)


In [16]:
for name, df in dfs.items():
    print(name, df.columns.tolist())


02-15-2018.csv ['dst_port', 'protocol', 'flow_duration', 'tot_fwd_pkts', 'tot_bwd_pkts', 'totlen_fwd_pkts', 'totlen_bwd_pkts', 'fwd_pkt_len_max', 'fwd_pkt_len_min', 'fwd_pkt_len_mean', 'fwd_pkt_len_std', 'bwd_pkt_len_max', 'bwd_pkt_len_min', 'bwd_pkt_len_mean', 'bwd_pkt_len_std', 'flow_byts/s', 'flow_pkts/s', 'flow_iat_mean', 'flow_iat_std', 'flow_iat_max', 'flow_iat_min', 'fwd_iat_tot', 'fwd_iat_mean', 'fwd_iat_std', 'fwd_iat_max', 'fwd_iat_min', 'bwd_iat_tot', 'bwd_iat_mean', 'bwd_iat_std', 'bwd_iat_max', 'bwd_iat_min', 'fwd_psh_flags', 'fwd_header_len', 'bwd_header_len', 'fwd_pkts/s', 'bwd_pkts/s', 'pkt_len_min', 'pkt_len_max', 'pkt_len_mean', 'pkt_len_std', 'pkt_len_var', 'fin_flag_cnt', 'syn_flag_cnt', 'rst_flag_cnt', 'psh_flag_cnt', 'ack_flag_cnt', 'urg_flag_cnt', 'ece_flag_cnt', 'down/up_ratio', 'pkt_size_avg', 'fwd_seg_size_avg', 'bwd_seg_size_avg', 'subflow_fwd_pkts', 'subflow_fwd_byts', 'subflow_bwd_pkts', 'subflow_bwd_byts', 'init_fwd_win_byts', 'init_bwd_win_byts', 'fwd_act

In [17]:
# Suppose dfs is your dict of DataFrames keyed by filename
common_cols = set(dfs[next(iter(dfs))].columns)  # start with first CSV's columns
for df in dfs.values():
    common_cols &= set(df.columns)  # intersect with each CSV's columns
common_cols = list(common_cols)  # convert back to list

# Then drop extra columns in each df
for name, df in dfs.items():
    dfs[name] = df[common_cols]



In [18]:
for name, df in dfs.items():
    print(name, df.shape)


02-15-2018.csv (1048575, 69)
02-21-2018.csv (1048575, 69)
02-22-2018.csv (1048575, 69)
03-01-2018.csv (331125, 69)
02-28-2018.csv (613104, 69)
02-16-2018.csv (1048575, 69)
03-02-2018.csv (1048575, 69)
02-14-2018.csv (1048575, 69)


In [19]:
train_days = [
    '02-14-2018.csv',
    '02-15-2018.csv',
    '02-16-2018.csv',
    '02-21-2018.csv',
    '02-22-2018.csv'
]

train_df = pd.concat([dfs[d] for d in train_days], ignore_index=True)

print(train_df.shape)


(5242875, 69)


In [20]:
# protocol to numeric
train_df['protocol'] = pd.to_numeric(train_df['protocol'], errors='coerce')

# label -> categorical index encoding
train_df['label'] = train_df['label'].astype('category').cat.codes

# verify all non-label columns numeric
for c in train_df.columns:
    if c != 'label':
        train_df[c] = pd.to_numeric(train_df[c], errors='coerce')

# final safety missing fill
train_df = train_df.fillna(train_df.median(numeric_only=True))


In [21]:
X = train_df.drop(columns=['label'])
y = train_df['label']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [22]:
# Convert y to a pandas Series if it isn't already
y_series = pd.Series(y)

# Find the 4 smallest classes
smallest_4 = y_series.value_counts().nsmallest(4).index
print("Dropping classes:", list(smallest_4))

# Create mask to keep only rows NOT in the rare classes
mask = ~y_series.isin(smallest_4)

# Filter X and y
X_filtered = X_scaled[mask]
y_filtered = y_series[mask]

# Optional: verify
print("New class distribution:")
print(y_filtered.value_counts())


Dropping classes: [10, 11, 2, 1]
New class distribution:
label
0     3519521
3      686012
6      461912
9      193360
12     187589
7      139890
5       41508
8       10990
4        1730
Name: count, dtype: int64


In [23]:
X_train, X_val, y_train, y_val = train_test_split(
   X_filtered,
   y_filtered,
   test_size=0.2,
   random_state=42,
   stratify=y_filtered
)
rf = RandomForestClassifier(
   n_estimators=150,
   max_depth=15,
   max_features='sqrt',
   min_samples_split=4,
   n_jobs=2,
   class_weight='balanced'
)

rf.fit(X_train, y_train)

# Evaluate
from sklearn.metrics import accuracy_score, classification_report

y_pred = rf.predict(X_val)

print("Accuracy:", accuracy_score(y_val, y_pred))
print(classification_report(y_val, y_pred))




Accuracy: 0.9827668590361687
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    703905
           3       1.00      1.00      1.00    137202
           4       1.00      1.00      1.00       346
           5       1.00      1.00      1.00      8302
           6       1.00      1.00      1.00     92382
           7       0.77      0.52      0.62     27978
           8       1.00      1.00      1.00      2198
           9       0.72      0.89      0.79     38672
          12       1.00      1.00      1.00     37518

    accuracy                           0.98   1048503
   macro avg       0.94      0.93      0.93   1048503
weighted avg       0.98      0.98      0.98   1048503



In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Initialize the model
lr = LogisticRegression(
    max_iter=1000,        # make sure it converges
    class_weight='balanced',  # handle any class imbalance
    n_jobs=-1             # use all cores for speed
)

# Train
lr.fit(X_train, y_train)

# Evaluate
y_pred_lr = lr.predict(X_val)
print("Logistic Regression Accuracy:", accuracy_score(y_val, y_pred_lr))
print(classification_report(y_val, y_pred_lr))


Logistic Regression Accuracy: 0.9773076471884201
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    703905
           3       0.99      1.00      0.99    137202
           4       1.00      1.00      1.00       346
           5       0.99      0.96      0.97      8302
           6       1.00      1.00      1.00     92382
           7       0.65      0.55      0.59     27978
           8       0.88      1.00      0.94      2198
           9       0.71      0.79      0.74     38672
          12       1.00      1.00      1.00     37518

    accuracy                           0.98   1048503
   macro avg       0.91      0.92      0.92   1048503
weighted avg       0.98      0.98      0.98   1048503

